In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

loading the data set and defining features and tarets

In [ ]:
file_path = r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\polynomial_regression_train.csv"
data = pd.read_csv(file_path)

data = data.drop(columns=['ID'])

X = data.drop(columns=['Target']).to_numpy()
y = data['Target'].to_numpy()

splitting the data set into training and testing data set and normalising the data 

In [ ]:
def train_test_split_manual(X, y, test_size=0.2, random_seed=42):
    np.random.seed(random_seed)
    indices = np.random.permutation(X.shape[0])
    test_size = int(X.shape[0] * test_size)
    test_indices, train_indices = indices[:test_size], indices[test_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2)
X_train_mean=X_train.mean()
X_train_std =X_train.std()
X_train=(X_train-X_train_mean)/X_train_std
X_test = (X_test-X_train_mean)/X_train_std

generating polynomial feaures

In [ ]:




def generate_polynomial_features(X, degree):
    n_samples, n_features = X.shape
    poly_features = np.ones((n_samples, 1))  # Bias term         
    for d in range(1, degree + 1):
        for combination in np.array(np.meshgrid(*[range(n_features) for _ in range(d)])).T.reshape(-1, d):
            poly_features = np.hstack((poly_features, np.prod(X[:, combination], axis=1).reshape(-1, 1)))
    return poly_features



defining gradient descent algorithm

In [ ]:
def gradient_descent(X, y, learning_rate, epochs):
    
    n_samples, n_features = X.shape
    coefficients = np.zeros(n_features)  
    cost_history = []  
    
    for _ in range(epochs):
        y_pred = X @ coefficients
        gradients = (1/ n_samples) * X.T @ (y_pred - y)
        coefficients -= learning_rate * gradients
        
        mse = (1/2)*np.mean((y - y_pred) ** 2)
        cost_history.append(mse)  
        
        
    
    return coefficients, cost_history

predicted the testing data test

In [ ]:
def predict_polynomial_regression(X, coefficients):
    return X @ coefficients

defining the metrics

In [ ]:
def r2_score(y_true, y_pred):
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)  
    ss_residual = np.sum((y_true - y_pred) **2)
    return 1 - (ss_residual / ss_total)                      


    



training the data set 

In [ ]:

degree = 4
learning_rate = 0.01
epochs = 1000
X_poly_train = generate_polynomial_features(X_train, degree)
X_poly_test = generate_polynomial_features(X_test, degree)


coefficients,cost_fucntion = gradient_descent(X_poly_train, y_train,  learning_rate, epochs)



y_train_pred = predict_polynomial_regression(X_poly_train, coefficients)
y_test_pred = predict_polynomial_regression(X_poly_test, coefficients)

print("final loss = ",cost_fucntion[-1])
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\nFinal Results:")

print("Training R²:", train_r2)
print("Testing R²:", test_r2)        

In [ ]:
print(coefficients.shape)
print(X_poly_train.shape)
plt.plot(range(epochs),cost_fucntion) 
plt.title("convergence")
plt.xlabel("iterations")
plt.ylabel("cost fucntion")
plt.show()


testing the data set 

In [ ]:
test_data = pd.read_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\polynomial_regression_test.csv")

X_new = test_data.drop(columns=["ID"]).values
X_new = (X_new - X_train_mean)/X_train_std 

X_poly_new = generate_polynomial_features(X_new, degree)
y_new_pred = predict_polynomial_regression(X_poly_new, coefficients)


test_data['Predicted'] = y_new_pred
test_data.to_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\polynomial_regression_test_pred.csv", index=False)

print("Predictions saved in the original test data file.")

